In [3]:
#Import relevant libraries
!pip install geopy
!pip install folium

import pandas as pd
import numpy as np

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import json

In [4]:
result = pd.read_csv('result.csv')
result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [5]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Latitude, Longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [6]:
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)

In [7]:
# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Here we can see all 103 Neighbourhoods of Toronto in the dataframe plotted onto a map. Purely from a visual reading we can say that there is a higher density of neighbourhoods clustered around the city center. Zooming in it is possible to see that these are in and around the financial district which would make sense in that cities are more densely populated around the center.

In [8]:
central_toronto_data = result[result['Borough'] == 'Central Toronto'].reset_index(drop=True)
downtown_toronto_data = result[result['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
west_toronto_data = result[result['Borough'] == 'West Toronto'].reset_index(drop=True)
east_toronto_data = result[result['Borough'] == 'East Toronto'].reset_index(drop=True)
east_toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [9]:
map_toronto_districts = folium.Map(location=[Latitude, Longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(central_toronto_data['Latitude'], central_toronto_data['Longitude'], central_toronto_data['Borough'], central_toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_districts)
for lat, lng, borough, neighborhood in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Borough'], downtown_toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_districts)
for lat, lng, borough, neighborhood in zip(west_toronto_data['Latitude'], west_toronto_data['Longitude'], west_toronto_data['Borough'], west_toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_districts)
for lat, lng, borough, neighborhood in zip(east_toronto_data['Latitude'], east_toronto_data['Longitude'], east_toronto_data['Borough'], east_toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_districts)
    
map_toronto_districts

This map shows the 'downtown' - green, 'central' - blue, 'west' - red and 'east' - purple boroughs of Toronto. The colouring allows the person viewing the map to see how they are divided geographically throughout the city. We can see that 'downtown' has the most dense clustering of boroughs in the city.